<a href="https://colab.research.google.com/github/Okitrader/freecodecamp_agloML_crypto/blob/main/FCCalgoML_crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Packages Needed:
pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

In [1]:
!pip install pandas warnings pandas_ta numpy matplotlib statsmodels pandas-datareader datetime yfinance scikit-learn PyPortfolioOpt


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [2]:
!pip install pandas_ta
!pip install selenium
!pip install requests

In [3]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')


In [22]:
# Import necessary libraries
import requests
import pandas as pd
from google.colab import userdata

# Retrieve API key from Colab user data
api_key = userdata.get('CMCKey')

if api_key is None:
    print("API key not found. Please check that the secret name is correct and that the notebook has access to it.")
else:
    print("API key retrieved successfully.")

# CoinMarketCap API URL and parameters
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'500',
  'convert':'USD'
}
headers = {
  'Accept': 'application/json',  # Corrected header
  'X-CMC_PRO_API_KEY': api_key,
}


# Make the API request
try:
    response = requests.get(url, headers=headers, params=parameters)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    data = response.json()

    # Parse the data to extract 'cmc_rank' and 'symbol'
    cryptos = [{'cmc_rank': entry['cmc_rank'], 'symbol': entry['symbol']} for entry in data['data']]

    # Create DataFrame and name it top_500_crypto
    top_500_crypto = pd.DataFrame(cryptos, columns=['cmc_rank', 'symbol'])

    # Display the DataFrame
    print(top_500_crypto.head())
except requests.exceptions.HTTPError as errh:
    print("Http Error:", errh)
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
except requests.exceptions.RequestException as err:
    print("Oops: Something Else", err)


API key retrieved successfully.
   cmc_rank symbol
0         1    BTC
1         2    ETH
2         3   USDT
3         4    BNB
4         5    XRP


In [23]:
print(top_500_crypto.tail(10))

     cmc_rank symbol
490       491    RKN
491       492   CHNG
492       493    FEI
493       494    VGX
494       495   BOND
495       496    ASD
496       497   RARE
497       498   POLS
498       499    UTK
499       502    AHT


In [24]:
import pandas as pd

# Get today's date as the end_date
end_date = pd.Timestamp('now').normalize()

# Subtract 8 years to get the start_date
start_date = end_date - pd.DateOffset(years=8)

print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")


Start Date: 2015-11-05 00:00:00
End Date: 2023-11-05 00:00:00


In [28]:
import requests
import pandas as pd
from google.colab import userdata

# Retrieve API key from Colab user data
api_key = userdata.get('CMCKey')


# CoinMarketCap API URL for historical quotes
url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/historical'

# Headers for the API request
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

# Minimal parameters for a single symbol
parameters = {
    'symbol': 'BTC',  # Use a valid cryptocurrency symbol
    'time_start': '2021-01-01',  # Use a valid start date
    'time_end': '2021-01-02',  # Use a valid end date
    # Don't include 'interval' or 'convert' for this test
}

# Make the API request for historical data
response = requests.get(url, headers=headers, params=parameters)

try:
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    data = response.json()
    print(data)  # Print data to inspect the structure
except requests.exceptions.HTTPError as err:
    # Print the HTTPError and the detailed error message from the API response
    print("HTTP Error:", err)
    print("Status Code:", response.status_code)
    try:
        error_details = response.json()
        print("Error details:", error_details)
    except ValueError:
        print("Error: Response is not in JSON format.")


HTTP Error: 400 Client Error: Bad Request for url: https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/historical?symbol=BTC&time_start=2021-01-01&time_end=2021-01-02
Status Code: 400
Error details: {'status': {'timestamp': '2023-11-05T12:35:04.871Z', 'error_code': 400, 'error_message': 'Your plan allows 1 months of historical access. Please upgrade your plan or choose a startDate that is newer than 2023-10-04T12:35:04.871Z.', 'elapsed': 0, 'credit_count': 0, 'notice': None}}


In [40]:
import yfinance as yf
import pandas as pd

# Set the dates for fetching historical data within the allowed range
end_date = pd.Timestamp('now').normalize()
start_date = end_date - pd.DateOffset(months=1)  # Adjusted to 1 month due to plan restrictions

# Assuming 'top_500_crypto' is a DataFrame containing a 'symbol' column
# with cryptocurrency symbols. We append '-USD' to each symbol to denote
# USD denominated trades.
symbols_list = [f"{symbol}-USD" for symbol in top_500_crypto['symbol']]

# Initialize an empty DataFrame to hold the data
df = pd.DataFrame()

# Download historical data for the given symbols
try:
    # yfinance might not have data for all crypto symbols, and the operation may take some time
    # Fetching the data in smaller batches if there are too many symbols might be necessary
    data = yf.download(tickers=symbols_list, start=start_date, end=end_date)

    # yfinance gives multi-level columns when downloading multiple tickers
    # We only want to keep 'Adj Close', 'Close', 'High', 'Low', 'Open', and 'Volume'
    # Check if the data has an 'Adj Close' level; if not, we'll skip it
    if 'Adj Close' in data.columns.levels[0]:
        cols_to_keep = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    else:
        cols_to_keep = ['Close', 'High', 'Low', 'Open', 'Volume']

    # Select the desired columns and stack the DataFrame to have a multiIndex with 'Date' and 'Ticker'
    df = data.loc[:, cols_to_keep].stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()
    df.columns = ['Date', 'Ticker'] + [col.lower().replace(' ', '_') for col in df.columns.tolist()[2:]]

except Exception as e:
    print(f"An error occurred: {e}")

# Show the head of the DataFrame
print(df.head())


[*********************100%%**********************]  498 of 498 completed

ERROR:yfinance:
52 Failed downloads:
ERROR:yfinance:['STT-USD', 'VIC-USD', 'SYN-USD', 'MCT-USD', 'SAITAMA-USD', 'BONE-USD', 'PLA-USD', 'LINA-USD', 'TOMI-USD', 'MASK-USD', 'HFT-USD', 'USDX-USD', 'LUNA-USD', 'IMX-USD', 'ONG-USD', 'AXL-USD', 'GMT-USD', 'SUPER-USD', 'MRS-USD', 'BFC-USD', 'MBX-USD', 'MDX-USD', 'PLANET-USD', 'FORT-USD', 'ALI-USD', 'MAGIC-USD', 'FCT-USD', 'GRT-USD', 'EDU-USD', 'PEPE-USD', 'COMP-USD', 'AURA-USD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-10-05 00:00:00 -> 2023-11-05 00:00:00)')
ERROR:yfinance:['EURC-USD', 'ARKM-USD', 'BORG-USD', 'SEI-USD', 'LYX-USD', 'LYXE-USD', 'VBUSD-USD', 'FDUSD-USD', 'FNSA-USD', 'WSM-USD', 'ORDI-USD', 'IOTA-USD', 'RON-USD', 'ETHDYDX-USD', 'PAAL-USD', 'PYUSD-USD', 'WEOS-USD']: IndexError('index 0 is out of bounds for axis 0 with size 0')
ERROR:yfinance:['STG-USD', 'TOKEN-USD', 'UNIBOT-USD']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



        Date     Ticker  adj_close      close       high        low  \
0 2023-10-05    0X0-USD   0.059155   0.059155   0.062358   0.057448   
1 2023-10-05  1INCH-USD   0.250272   0.250272   0.256638   0.250101   
2 2023-10-05   AAVE-USD  66.832779  66.832779  72.130043  66.298157   
3 2023-10-05   ABBC-USD   0.031821   0.031821   0.033176   0.031701   
4 2023-10-05    ACA-USD   0.050212   0.050212   0.053633   0.050036   

        open       volume  
0   0.062097     638489.0  
1   0.256005    9651916.0  
2  66.614746  121768895.0  
3   0.033146    4921289.0  
4   0.053245    3062315.0  


In [41]:
df.tail()

,Date,Ticker,adj_close,close,high,low,open,volume
13816,2023-11-04,YGG-USD,0.286965,0.286965,0.298816,0.278942,0.279673,36254183.0
13817,2023-11-04,ZEC-USD,28.210516,28.210516,28.251926,27.613274,27.637863,48513237.0
13818,2023-11-04,ZEN-USD,9.429918,9.429918,9.441409,9.156373,9.377581,6356179.0
13819,2023-11-04,ZIL-USD,0.020233,0.020233,0.020352,0.019440,0.019532,14544392.0
13820,2023-11-04,ZRX-USD,0.276462,0.276462,0.277578,0.263067,0.267628,22203589.0


In [43]:
df.columns = df.columns.str.lower()
df.head()

,date,ticker,adj_close,close,high,low,open,volume
0,2023-10-05,0X0-USD,0.059155,0.059155,0.062358,0.057448,0.062097,638489.0
1,2023-10-05,1INCH-USD,0.250272,0.250272,0.256638,0.250101,0.256005,9651916.0
2,2023-10-05,AAVE-USD,66.832779,66.832779,72.130043,66.298157,66.614746,121768895.0
3,2023-10-05,ABBC-USD,0.031821,0.031821,0.033176,0.031701,0.033146,4921289.0
4,2023-10-05,ACA-USD,0.050212,0.050212,0.053633,0.050036,0.053245,3062315.0


In [36]:
df.stack()
df.index.names = ['date']
print(df)

            Date     Ticker        0x0-usd     1inch-usd      aave-usd  \
date                                                                     
0     2015-09-27  Adj Close            NaN           NaN           NaN   
1     2015-09-27      Close            NaN           NaN           NaN   
2     2015-09-27       High            NaN           NaN           NaN   
3     2015-09-27        Low            NaN           NaN           NaN   
4     2015-09-27       Open            NaN           NaN           NaN   
...          ...        ...            ...           ...           ...   
17527 2023-09-26      Close       0.041990  2.666520e-01  6.108453e+01   
17528 2023-09-26       High       0.042013  2.696230e-01  6.227058e+01   
17529 2023-09-26        Low       0.037904  2.622750e-01  6.060524e+01   
17530 2023-09-26       Open       0.037908  2.644950e-01  6.227058e+01   
17531 2023-09-26     Volume  580988.000000  1.505585e+07  5.829256e+07   

           abbc-usd       aca-usd    

In [ ]:
# import requests
# import pandas as pd
# from google.colab import userdata

# # Retrieve API key from Colab user data
# api_key = userdata.get('CMCKey')

# # CoinMarketCap API URL for historical quotes
# url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/historical'

# # Set the dates for fetching historical data within the allowed range
# end_date = pd.Timestamp('now').normalize()
# start_date = end_date - pd.DateOffset(months=1)  # Adjusted to 1 month due to plan restrictions

# # Headers for the API request
# headers = {
#     'Accepts': 'application/json',  # Changed from 'Accept' to 'Accepts' which is the correct header
#     'X-CMC_PRO_API_KEY': api_key,
# }

# # Initialize an empty DataFrame for all the historical data
# crypto500_OHLVC = pd.DataFrame()

# # Assume 'top_500_crypto' is a DataFrame containing the 'symbol' column
# for symbol in top_500_crypto['symbol']:
#     parameters = {
#         'symbol': symbol,
#         'time_start': start_date.isoformat(),
#         'time_end': end_date.isoformat(),
#         'interval': '1d',  # Daily intervals
#         'convert': 'USD'
#     }

#     try:
#         # Make the API request for historical data
#         response = requests.get(url, headers=headers, params=parameters)
#         response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
#         data = response.json()

#         # Check if 'quotes' is in data['data']
#         if 'quotes' in data['data']:
#             # Process the response data and extract OHLC
#             quotes = data['data']['quotes']
#             ohlc_data = []
#             for quote in quotes:
#                 timestamp = quote['timestamp']
#                 price_info = quote['quote']['USD']
#                 ohlc_data.append({
#                     'symbol': symbol,
#                     'date': pd.to_datetime(timestamp),
#                     'open': price_info.get('open', None),  # Assuming these fields are provided
#                     'high': price_info.get('high', None),
#                     'low': price_info.get('low', None),
#                     'close': price_info.get('close', None),
#                     'volume': price_info['volume_24h']

#                 })

#             # Convert to DataFrame and append to the main DataFrame
#             symbol_ohlc_df = pd.DataFrame(ohlc_data)
#             crypto500_OHLVC = pd.concat([crypto500_OHLVC, symbol_ohlc_df], ignore_index=True)
#         else:
#             print(f"No 'quotes' data for symbol {symbol}")

#     except requests.exceptions.HTTPError as e:
#         print(f"HTTPError for symbol {symbol}: {e}")

# # Set the proper multi-index for date and ticker
# crypto500_OHLVC.set_index(['date', 'symbol'], inplace=True)

# # Sort the DataFrame (optional, for better organization)
# crypto500_OHLVC.sort_index(inplace=True)

# # Show the DataFrame
# print(crypto500_OHLVC.head())
